In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

data_path = "./data/toxic_spans"
filename = "toxic_span_text_pairs.csv"
splits = ["train", "eval", "test"]

dataset_dict = DatasetDict(
    {
        split: Dataset.from_pandas(
            pd.read_csv(
                data_path + "/" + filename.replace(".csv", "_" + split + ".csv")
            )
        )
        for split in splits
    }
)

In [ ]:
import torch


def detox(tokenizer, model_buffered, batched_inputs):
    input_sequence_buffered = tokenizer(
        batched_inputs, padding="longest", pad_to_multiple_of=8, return_tensors="pt"
    ).to("cuda")
    with torch.no_grad():
        output_sequence = model_buffered.generate(
            input_ids=input_sequence_buffered["input_ids"],
            attention_mask=input_sequence_buffered["attention_mask"],
        )
    del input_sequence_buffered
    generated = [
        "<CSD>" if len(generated) == 0 else generated
        for generated in tokenizer.batch_decode(
            output_sequence, skip_special_tokens=True
        )
    ]
    return {"generated": generated}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
from datasets import Dataset

for model_name in ["t5-base", "t5-v1_1-base", "bart-base"]:
    model_name = f"ChrisZeng/{model_name}-detox"

    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_buffered = model.to("cuda")
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token

    dataset_dict = dataset_dict.map(
        lambda rec: detox(tokenizer, model_buffered, rec["original"]),
        keep_in_memory=True,
        batched=True,
        batch_size=64,
    ).rename_columns({"generated": f"generated_{model_name}"})

    del model_buffered

In [ ]:
import re


class SentenceDetoxer(object):
    def __init__(self, toxic_words):
        self.replacements = {toxic_word: "<CSD>" for toxic_word in toxic_words}
        self.pattern = "|".join(r"\b%s\b" % re.escape(s) for s in self.replacements)

    def __call__(self, sentence):
        return re.sub(
            self.pattern, lambda match: self.replacements[match.group(0)], sentence
        )


In [ ]:
import pandas as pd

hatebase = pd.read_csv("outputs/hatebase.csv")
hate_words = hatebase[hatebase["language"] == "English"]["word"]
hate_words = hate_words.str.split("(").str[0].str.strip()
toxic_words = pd.concat(
    [hate_words, pd.read_csv("outputs/bad-words.csv")["word"],]
).drop_duplicates()

detoxer = SentenceDetoxer(toxic_words)

dataset_dict = dataset_dict.map(
    lambda record: {"naive-detox": detoxer(record["original"])}
)


In [ ]:
dataset_dict

In [ ]:
for split, dataset in dataset_dict.items():
    dataset.to_pandas().to_csv(f"outputs/generated_outputs_{split}.csv", index=False)


In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict

dataset_dict = DatasetDict(
    {
        split: Dataset.from_pandas(
            pd.read_csv(f"outputs/generated_outputs_{split}.csv")
        )
        for split in ["train", "eval", "test"]
    }
)



In [4]:
from transformers import pipeline
import pandas as pd
import numpy as np


class ToxicScore(object):
    def __init__(self, score_pipeline):
        self.score_pipeline = score_pipeline

    def __call__(self, inputs):
        return np.mean([result[0]["score"] for result in detoxify_pipeline(inputs)])


detoxify_pipeline = pipeline(
    "text-classification",
    model="unitary/toxic-bert",
    tokenizer="bert-base-uncased",
    function_to_apply="sigmoid",
    return_all_scores=True,
    device=0,
    batch_size=8,
)

toxicscore = ToxicScore(detoxify_pipeline)

from datasets import load_metric
bertscore = load_metric("bertscore")



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [5]:
from datasets import load_metric
import numpy as np
import pandas as pd
from IPython.display import display, Pretty

rouge = load_metric("rouge")
exact_match = load_metric("exact_match")
bertscore = load_metric("bertscore")


def compute_metrics(original, generated):
    return {
        "similarity": np.mean(
            bertscore.compute(predictions=generated, references=original, lang="en")[
                "f1"
            ]
        ),
        "toxicity": np.mean(toxicscore(generated)),
    }


metrics = pd.concat(
    {
        split: pd.concat(
            pd.DataFrame(
                compute_metrics(
                    dataset_dict[split]["original"], dataset_dict[split][col]
                ),
                index=[col],
            )
            for col in dataset_dict[split].column_names
        )
        for split in dataset_dict.keys()
    },
    names=["split", "column"],
)

metrics


/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using 

similarity  toxicity
split column                                                      
train original                                  1.000005  0.634264
      censored                                  0.938402  0.119891
      generated_ChrisZeng/t5-base-detox         0.904883  0.117076
      generated_ChrisZeng/t5-v1_1-base-detox    0.909105  0.143238
      generated_ChrisZeng/bart-base-detox       0.850838  0.077703
      naive-detox                               0.978006  0.396886
eval  original                                  1.000006  0.638489
      censored                                  0.939461  0.120043
      generated_ChrisZeng/t5-base-detox         0.906586  0.120961
      generated_ChrisZeng/t5-v1_1-base-detox    0.910119  0.143255
      generated_ChrisZeng/bart-base-detox       0.852211  0.081079
      naive-detox                               0.978564  0.391680
test  original                                  1.000004  0.629271
      censored                                  0.939285  0.111152
      generated_ChrisZeng/t5-base-detox         0.906485  0.112214
      generated_ChrisZeng/t5-v1_1-base-detox    0.909313  0.133200
      generated_ChrisZeng/bart-base-detox       0.855033  0.081856
      naive-detox                               0.978401  0.386676

In [6]:
from datasets import load_metric
import numpy as np
import pandas as pd
from IPython.display import display, Pretty

rouge = load_metric("rouge")
exact_match = load_metric("exact_match")
bertscore = load_metric("bertscore")


def compute_metrics(predictions, targets):
    return {
        **{
            key: value.mid.fmeasure
            for key, value in rouge.compute(
                predictions=predictions, references=targets
            ).items()
        },
        "exact_match_rate": 0.01
        * exact_match.compute(predictions=predictions, references=targets)[
            "exact_match"
        ],
        "mean_bertscore_f1": np.mean(
            bertscore.compute(predictions=predictions, references=targets, lang="en")[
                "f1"
            ]
        ),
    }


metrics = pd.concat(
    {
        split: pd.concat(
            pd.DataFrame(
                compute_metrics(
                    dataset_dict[split][col], dataset_dict[split]["censored"]
                ),
                index=[col],
            )
            for col in [
                "generated_ChrisZeng/t5-base-detox",
                "generated_ChrisZeng/t5-v1_1-base-detox",
                "generated_ChrisZeng/bart-base-detox",
                "naive-detox",
            ]
        )
        for split in dataset_dict.keys()
    },
    names=["split", "column"],
)

metrics


/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


rouge1    rouge2    rougeL  \
split column                                                                 
train generated_ChrisZeng/t5-base-detox       0.600023  0.527782  0.596384   
      generated_ChrisZeng/t5-v1_1-base-detox  0.573877  0.499064  0.569643   
      generated_ChrisZeng/bart-base-detox     0.326864  0.160558  0.323464   
      naive-detox                             0.813105  0.742662  0.808225   
eval  generated_ChrisZeng/t5-base-detox       0.604262  0.523416  0.599722   
      generated_ChrisZeng/t5-v1_1-base-detox  0.584831  0.506420  0.580486   
      generated_ChrisZeng/bart-base-detox     0.332699  0.166041  0.328492   
      naive-detox                             0.817980  0.747777  0.812850   
test  generated_ChrisZeng/t5-base-detox       0.593510  0.514044  0.589424   
      generated_ChrisZeng/t5-v1_1-base-detox  0.574478  0.496127  0.570753   
      generated_ChrisZeng/bart-base-detox     0.339363  0.176767  0.335875   
      naive-detox                             0.813931  0.744925  0.809108   

                                              rougeLsum  exact_match_rate  \
split column                                                                
train generated_ChrisZeng/t5-base-detox        0.596429          0.028920   
      generated_ChrisZeng/t5-v1_1-base-detox   0.569903          0.091600   
      generated_ChrisZeng/bart-base-detox      0.324277          0.051734   
      naive-detox                              0.808793          0.103583   
eval  generated_ChrisZeng/t5-base-detox        0.599935          0.020323   
      generated_ChrisZeng/t5-v1_1-base-detox   0.580810          0.080323   
      generated_ChrisZeng/bart-base-detox      0.329628          0.043871   
      naive-detox                              0.813122          0.109355   
test  generated_ChrisZeng/t5-base-detox        0.589320          0.016129   
      generated_ChrisZeng/t5-v1_1-base-detox   0.570676          0.079839   
      generated_ChrisZeng/bart-base-detox      0.336405          0.042473   
      naive-detox                              0.809765          0.112903   

                                              mean_bertscore_f1  
split column                                                     
train generated_ChrisZeng/t5-base-detox                0.915085  
      generated_ChrisZeng/t5-v1_1-base-detox           0.912732  
      generated_ChrisZeng/bart-base-detox              0.882190  
      naive-detox                                      0.948868  
eval  generated_ChrisZeng/t5-base-detox                0.914312  
      generated_ChrisZeng/t5-v1_1-base-detox           0.913919  
      generated_ChrisZeng/bart-base-detox              0.882423  
      naive-detox                                      0.950081  
test  generated_ChrisZeng/t5-base-detox                0.913068  
      generated_ChrisZeng/t5-v1_1-base-detox           0.912746  
      generated_ChrisZeng/bart-base-detox              0.883306  
      naive-detox                                      0.949808